In [1]:
import transformers
import torch

model = transformers.AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m-deduped")
print(model)

/home/xzyao/miniconda3/envs/fmsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-07-25 16:11:17,785] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, o

In [ ]:
import transformers
import torch

model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
print(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.51s/it]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [ ]:
import transformers
import torch
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1
)

model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
print("standard model")
print(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
print(model.base_model.model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


standard model
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (no

In [6]:
import torch
tensor = torch.load("../.cache/models-mp/global_step5/layer_02-model_00-model_states.pt")
print(tensor.keys())

odict_keys(['self_attn.q_proj.weight', 'self_attn.q_proj.lora_A', 'self_attn.q_proj.lora_B', 'self_attn.k_proj.weight', 'self_attn.v_proj.weight', 'self_attn.v_proj.lora_A', 'self_attn.v_proj.lora_B', 'self_attn.o_proj.weight', 'mlp.gate_proj.weight', 'mlp.down_proj.weight', 'mlp.up_proj.weight', 'input_layernorm.weight', 'post_attention_layernorm.weight'])


In [10]:
from safetensors import safe_open

with safe_open("../.cache/hf/model.safetensors", framework="pt", device="cpu") as f:
    total_numel = 0
    for key in f.keys():
        total_numel += f.get_tensor(key).numel()
        print(f"{key} - {f.get_tensor(key).shape}")
print(f"Total number of parameters: {total_numel}")

lm_head.weight - torch.Size([32000, 3200])
model.embed_tokens.weight - torch.Size([32000, 3200])
model.layers.0.input_layernorm.weight - torch.Size([3200])
model.layers.0.mlp.down_proj.weight - torch.Size([3200, 8640])
model.layers.0.mlp.gate_proj.weight - torch.Size([8640, 3200])
model.layers.0.mlp.up_proj.weight - torch.Size([8640, 3200])
model.layers.0.post_attention_layernorm.weight - torch.Size([3200])
model.layers.0.self_attn.k_proj.lora_A - torch.Size([32, 3200])
model.layers.0.self_attn.k_proj.lora_B - torch.Size([3200, 32])
model.layers.0.self_attn.o_proj.lora_A - torch.Size([32, 3200])
model.layers.0.self_attn.o_proj.lora_B - torch.Size([3200, 32])
model.layers.0.self_attn.q_proj.lora_A - torch.Size([32, 3200])
model.layers.0.self_attn.q_proj.lora_B - torch.Size([3200, 32])
model.layers.0.self_attn.v_proj.lora_A - torch.Size([32, 3200])
model.layers.0.self_attn.v_proj.lora_B - torch.Size([3200, 32])
model.layers.1.input_layernorm.weight - torch.Size([3200])
model.layers.1.mlp